In [1]:
! pip install torch trasnformers

ERROR: Could not find a version that satisfies the requirement trasnformers (from versions: none)
ERROR: No matching distribution found for trasnformers


In [3]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [4]:

# 1. Carica modello e tokenizer
model_name = "gpt2"  # Puoi usare anche "gpt2-medium"
model = GPT2LMHeadModel.from_pretrained(model_name, output_attentions=True)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model.eval()

The following generation flags are not valid and may be ignored: ['output_attentions']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['output_attentions']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['output_attentions']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['output_attentions']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [5]:

# 2. Prompt
prompt = "Write a short fairy tale combining horror and comedy."
inputs = tokenizer(prompt, return_tensors="pt")

In [ ]:
# 3. Generazione con output di attentions e logits
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_length=80,
        return_dict_in_generate=True,
        output_scores=True,
        output_attentions=True
    )

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
`torch.nn.functional.scaled_dot_product_attention` does not support `output_attentions=True`. Falling back to eager attention. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


In [ ]:

# 4. Estrai token, logits, attentions
generated_ids = outputs.sequences[0]
generated_tokens = tokenizer.convert_ids_to_tokens(generated_ids)

# Logits → logprobs
scores = outputs.scores  # list of [1, vocab_size]
logprobs = [torch.log_softmax(s, dim=-1) for s in scores]
topk_probs = [lp.topk(5) for lp in logprobs]

# Attentions
attentions = outputs.attentions  # list of tuples per layer → (1, heads, tgt_len, src_len)

In [ ]:

# 5. Stampa token + logprobs del token scelto
print("Generated text:\n", tokenizer.decode(generated_ids))
print("\nLogprobs of chosen tokens:")
for i, (token_id, lp) in enumerate(zip(generated_ids[len(inputs["input_ids"][0]):-1], logprobs)):
    token = tokenizer.decode([token_id])
    logp = lp[0, token_id].item()
    print(f"{i+1}: {token.strip()} → logprob = {logp:.3f}")

In [2]:

# 6. Visualizza heatmap di attenzione (ultimo layer, prima head)
last_layer_attention = attentions[-1][0][0].numpy()  # shape: (tgt_len, src_len)

plt.figure(figsize=(12, 8))
sns.heatmap(last_layer_attention[:len(generated_ids), :len(generated_ids)],
            xticklabels=generated_tokens,
            yticklabels=generated_tokens,
            cmap="viridis", annot=False)
plt.title("Attention heatmap (last layer, head 0)")
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

/opt/anaconda3/envs/meta-prompting/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'matplotlib'